<div style="text-align: center;">
  <h1>TA136 - Taller de Procesamiento de Señales</h1>
  <h2>Trabajo Práctico 9: Modelo de Lenguaje y Sistema de Recomendación</h2>
</div>

---
---

<div style="text-align: center;">
  <h3> Introducción
</div>

&ensp; El presente trabajo práctico tiene como objetivo el diseño de un sistema de recomendación de películas, complementado con un modelo de lenguaje que permita buscar títulos aún si son ingresados con errores. El desarrollo se estructura en dos incisos principales: el buscador de películas y el sistema de recomendación.

&ensp; En la primera etapa, se construye un modelo de lenguaje utilizando representaciones pre-entrenadas con $\texttt{GloVe}$ y dimensión $300$. A partir de este modelo, se implementan funciones que permiten obtener vectores de palabras, formar representaciones de oraciones mediante una bolsa de palabras y calcular similitudes entre representaciones usando la similitud coseno. Esto, permite implementar un buscador capaz de sugerir títulos cercanos a lo que el usuario ingresa, incluso ante errores o imprecisiones.

&ensp; Por otro lado, en la segunda etapa, se desarrolla un sistema de recomendación utilizando un filtro colaborativo. Para ello, se agrega un nuevo usuario con más de diez películas calificadas, y se entrena un modelo con *gradient descent*, usando un espacio latente de dimensión $10$ y regularización. Finalmente, se combinan los resultados del filtro colaborativo con la calificación promedio de las películas para obtener un *rating*, con el que se realiza una recomendación de cinco películas no vistas.

---
---

<div style="text-align: center;">
  <h3> Desarrollo
</div>

**Se desea crear un sistema para recomendar películas. El archivo $\texttt{movies.csv}$ posee una base de datos donde usuarios calificaron (del $1$ al $5$) diferentes películas ($0$ significa sin calificar).**

---

#### (A) *Modelo de Lenguaje:*

**Se desea diseñar un buscador de títulos de películas, de manera que si el usuario comete algún error u omisión cuando lo escribe, el buscador pueda entender. Para ello, se estudiará la similitud entre los *embeddings*. A continuación se describen los pasos para diseñar el buscador; se recomienda que los mismos sean métodos dentro de la clase del buscador mencionado.**

- **Descargar las representaciones pre-entrenadas $\texttt{GloVe}$ de dimensión $300$.**

- **Cargar el modelo de lenguaje.**

- **Implementar un *word2vec*. Si la palabra está en el vocabulario debe devolver el vector del modelo de lenguaje, caso contrario debe devolver un vector de ceros.**

- **Implementar una bolsa de palabras que transforme cualquier *string* en un vector. Los pasos a seguir son:**
    
    - **Convertir las mayúsculas en minúsculas.**
    - **Eliminar caracteres extraños.**
    - **Unificar espacios en blanco.**
    - **Convertir el *string* en una lista de palabras.**
    - **Convertir cada palabra en un *embedding* usando el *word2vec*.**
    - **Sumar las representaciones para formar un solo vector.**

- **Se desea medir que tan parecidos son dos *embeddings*. Para ello, implementar un código que calcule la similitud coseno.**

- **Implementar un buscador que, dado un *string* (y su correspondiente *embedding*), devuelva la película con una representación más similar.**

&ensp; A fin de desarrollar el buscador de títulos de películas, se siguieron los lineamientos dados por la cátedra y se definió la clase llamada $\texttt{LanguageModel}$. Esta implementa los métodos que se detallan a continuación:

- `__init__:` Inicializa la clase y declara los atributos necesarios para almacenar los parámetros del modelo, tales como la dirección *url* y el nombre del archivo que contiene las representaciones preentrenadas de palabras con $\texttt{GloVe}$, un diccionario para almacenar los *embeddings* de cada palabra y una lista con los *embeddings* correspondientes a los títulos de las películas. Durante esta inicialización, se ejecutan métodos internos del modelo: $\texttt{\_\_download\_model}$, $\texttt{\_\_load\_model}$ y $\texttt{\_\_calculate\_embedding\_titles()}$; los cuales se explican a continuación.

- `__download_model` y `__load_model:` Descarga las representaciones de $\texttt{GloVe}$ con dimensión $300$ y las carga al diccionario inicializado anteriormente.

- `word2vec:` Retorna el vector asociado a una palabra dada. Si la palabra no se encuentra en el modelo, se devuelve un vector nulo de dimensión $300$.
  
- `BoW:` Devuelve el *embedding* correspondiente a una *string* pasada por parámetro. Para ello, se siguen los pasos:
  1. Convertir el texto a minúsculas.
  2. Eliminar los caractéres que no son letras o espacios.
  3. Separar las palabras que componen a la *string*.
  4. Obtener el *embedding* correspondiente a cada palabra mediante $\texttt{word2vec}$.
  5. Sumar los vectores de cada palabra, para así obtener el *embedding* de la cadena.

- `sim_cos:` Calcula la similitud coseno entre dos *embeddings*. Para evitar divisiones por cero, si el denominador es nulo, se reemplaza por un valor mínimo $\epsilon = 10^{-8}$.

$$SC(u, ~v) = \frac{u \cdot v}{\texttt{max(}||u|| \cdot ||v||, ~ \epsilon\texttt{)}}$$

- `__calculate_embedding_titles:` Genera y almacena los *embeddings* de todos los títulos de la base de datos a partir de la bolsa de palabras implementada.

- `searcher:` Dado un título ingresado por el usuario, calcula su *embedding* y lo compara con los títulos de la base de datos mediante la similitud coseno. Luego, devuelve el título con mayor *SC*.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re

In [ ]:
class LanguageModel:
    def __init__(self, titles):
        self.titles = titles
        self.url = "http://nlp.stanford.edu/data/glove.6B.zip"
        self.filename = "glove.6B.300d.txt"
        self.language_model = {}
        self.embedding_titles = []
        self.__download_model()
        self.__load_model()
        self.__calculate_embedding_titles()

    def __download_model(self):
        if not os.path.exists(self.filename):
            print("Descargando GloVe...")
            os.system(f"wget {self.url}")
            os.system("unzip glove.6B.zip")

    def __load_model(self):
        with open(self.filename, encoding="utf-8") as f:
            for line in f:
                parts = line.strip().split()
                word = parts[0]
                vec = np.array(parts[1:], dtype=float)
                self.language_model[word] = vec

    def word2vec(self, word):
        return self.language_model.get(word, np.zeros(300))

    def BoW(self, string):
        low_string = string.lower()
        words = re.sub(r"[^a-z0-9 ]", "", low_string).split()
        embeddings = [self.word2vec(word) for word in words]
        return sum(embeddings)

    def sim_cos(self, embedding1, embedding2):
        return np.dot(embedding1, embedding2) / np.max([np.linalg.norm(embedding1) * np.linalg.norm(embedding2), 1e-8])

    def __calculate_embedding_titles(self):
        for title in self.titles:
            self.embedding_titles.append(self.BoW(title))

    def searcher(self, string):
        embedding = self.BoW(string)
        scores = [self.sim_cos(embedding, self.embedding_titles[i]) for i in range(len(self.embedding_titles))]
        return self.titles[np.argmax(scores)]


&ensp; Una vez implementada la clase, se obtiene el dataset del github de la materia y se lo baja como un *dataframe* de la librería `pandas`.

In [ ]:
path = "https://raw.githubusercontent.com/mvera1412/TA136-TB056-TB057-8625/main/data/movies.csv"
DF = pd.read_csv(path)
titulos = DF.Name.tolist()

&ensp; Luego, se inicializa la clase con los títulos extraídos del *dataframe* y se verifica el correcto funcionamiento del método correspondiente al buscador.

In [ ]:
LM = LanguageModel(titles=titulos)
LM.searcher("chocolate")

---

#### (B). *Sistemas de Recomendación:*

**Se desea diseñar el sistema de recomendación y utilizarlo para recomendarnos películas.**

- **Agregar un usuario a la base de datos con al menos $10$ películas calificadas. Utilice el buscador para no tener que escribir los títulos perfectos.**

- **Utilizando gradiente descendente entrenar un filtro colaborativo con un espacio latente de dimensión $10$, $\lambda = 10$ y *learning rate* $10^{−3}$. Graficar el riesgo regularizado empírico en función del número de iteraciones (al menos $2000$).**

- **Crear un *rating* ponderando en partes iguales la salida del filtro colaborativo y la calificación media de las películas.**

- **Recomendar las $5$ películas no vistas con más alto *rating* al usuario creado anteriormente.**

&ensp; Para desarrollar este apartado del trabajo, se implementa nuevamente una clase llamada $\texttt{RecommenderSystem}$, cuyo objetivo es definir un sistema de recomendación según lo visto en las clases teóricas de la materia.

&ensp; Dentro de los métodos de la clase, se encuentran los siguientes:

- `init:` Inicializa la clase y declara los atributos necesarios para almacenar las variables del modelo, entre las que se encuentran: el *dataframe* de las películas, las clasificaciones de los usuarios para cada película, una lista para almacenar el costo en cada iteración del entrenamiento y los parámetros de entrenamiento como $x$ y $\theta$.

- `add_user:` Agrega un usuario a la base de datos pasada como parámetro en la inicialización. Para ello, se utiliza la clase $\texttt{LanguageModel}$, que mejora el entendimiento de los títulos pasados. Además, en base a los títulos y los puntajes que se le pasan al método, se actualiza el *dataframe* con los *scores* correspondientes a cada película.

- `train_collab_filter:` Calcula los parámetros $x \in \mathbb{R}^{n_\text{items} \times k}$ y $\theta \in \mathbb{R}^{n_\text{users} \times k}$ del filtro colaborativo. A fin de realizar esto, se comienza planteando que se quiere minimizar la siguiente expresión:
$$\min_{x, ~\theta} \underbrace{\frac{1}{2} \sum_{(i, ~j): ~ y_{i, ~j} > 0} \left( \theta_j^T \cdot x_i - y_{i, ~j} \right)^2 + \frac{\lambda}{2} \left( \sum_{i = 1}^{n_{\text{items}}} ||x_i||^2 +  \sum_{j = 1}^{n_{\text{users}}} ||\theta_j||^2 \right)}_{J(x_i, ~\theta_j)};$$
donde $y_{i, ~j} \in \mathbb{R}^{n_{\text{items}} \times n_{\text{users}}}$ corresponde a la clasificación del *dataset* en la fila $i$ columna $j$, $k$ es la dimensión del espacio latente y $\lambda$ es un hiperparámetro de regularización. Así, se plantean las derivadas parciales de $J(x_i, ~\theta_j)$, tal que:
\begin{align*}
\begin{cases}
  \frac{\partial}{\partial x_i} J(x_i, ~ \theta_j) &= \sum_{j: ~ y_{i, ~j} > 0} \left( \theta_j^T \cdot x_i - y_{i, ~j} \right) \cdot \theta_j + \lambda \cdot x_i \\
  \frac{\partial}{\partial \theta_j} J(x_i, ~ \theta_j) &= \sum_{i: ~ y_{i, ~j} > 0} \left( \theta_j^T \cdot x_i - y_{i, ~j} \right) \cdot x_i + \lambda \cdot \theta_j
\end{cases}
\end{align*}
De esta manera, se obtienen los gradientes de cada una de las variables a determinar, por lo tanto, como se trabaja con el método del gradiente descendente, se tiene que los parámetros de entrenamiento que minimizan la función costo están dados por:
\begin{align*}
\begin{cases}
  x_{i, ~ t}  = x_{i, ~ t - 1} - \alpha \cdot \frac{\partial}{\partial x_i} J(x_i, ~ \theta_j) \\
  \theta_{j, ~ t}  = \theta_{j, ~ t - 1} - \alpha \cdot \frac{\partial}{\partial \theta_j} J(x_i, ~ \theta_j)
\end{cases}
\end{align*}
Además, el método calcula el costo según la expresión de $J(x_i, ~ \theta_j)$ en cada iteración y lo almacena en la lista inicializada anteriormente.

- `rating_ponderation:` Computa un *rating* considerando la salida del filtro colaborativo y la calificación media de las películas, con una proporción dada por el hiperparámetro $0 \leq p \leq 1$. Esto, se realiza a partir de la siguiente expresión:
$$\hat{y}_{i, ~ j} = p \left(\theta_j^T \cdot x_i \right) + (1 - p) \cdot \tilde{y}_i;$$
donde $\tilde{y}_i$ es la calificación promedio del $i$-ésimo item.

- `recommend:` A partir de los *ratings* estimados para el usuario sobre las películas disponibles, se recomienda un conjunto de $n$ películas no vistas que presentan los valores de *rating* más altos.

In [ ]:
class RecommenderSystem:
    def __init__(self, dataframe):
        if "Name" not in dataframe.columns:
            raise ValueError("El DataFrame debe contener una columna 'Name'.")

        self.df = dataframe
        self.y = None
        self.y_qualified = None
        self.x = None
        self.theta = None
        self.cost = []

    def add_user(self, language_model, new_user, movies, scores):
        if len(movies) != len(scores):
          raise ValueError("La cantidad de películas y puntajes debe coincidir.")

        self.df[new_user] = 0
        for i in range(len(movies)):
          best_title = language_model.searcher(movies[i])
          self.df.loc[self.df['Name'] == best_title, new_user] = scores[i]
        return self.df.loc[self.df[new_user] != 0, ['Name', new_user]]

    def train_collab_filter(self, learning_rate, lambda_, k, n_iter):
        self.y = self.df.drop(columns=["Name"]).values.astype(float)
        self.y_qualified = (self.y > 0).astype(int)
        n_items, n_users = self.y.shape

        self.x = np.random.normal(loc=0, scale=0.1, size=(n_items, k))
        self.theta = np.random.normal(loc=0, scale=0.1, size=(n_users, k))

        for i in range(n_iter):
          error = (self.x @ self.theta.T - self.y) * self.y_qualified
          x_gradient = error @ self.theta + lambda_ * self.x
          theta_gradient = error.T @ self.x + lambda_ * self.theta

          self.x -= learning_rate * x_gradient
          self.theta -= learning_rate * theta_gradient

          cost_aux = 0.5 * np.sum(error**2) + (lambda_ / 2) * (np.sum(self.x**2) + np.sum(self.theta**2))
          self.cost.append(cost_aux)

    def rating_ponderation(self, p):
        sum_scores = np.sum(self.y, axis=1, keepdims=True)
        count_scores = np.sum(self.y_qualified, axis=1, keepdims=True)
        y_mean_score =  np.divide(sum_scores, count_scores, out=np.zeros_like(sum_scores), where=count_scores!=0)

        return p * (self.x @ self.theta.T) + (1 - p) * y_mean_score

    def recommend(self, user, n, p):
        if user not in self.df.columns:
          raise ValueError("El usuario no está en la base de datos.")

        idx_user = self.df.columns.get_loc(user) - 1
        idx_unseen = np.where(self.y_qualified[:, idx_user] == 0)[0]
        ratings_user = self.rating_ponderation(p)[idx_unseen, idx_user]

        top_movies = idx_unseen[np.argsort(ratings_user)[::-1][:n]]
        return self.df.iloc[top_movies]['Name']

&ensp; Una vez implementada la clase, se la inicializa y se agrega el usuario `Felipe` con puntuaciones de $14$ películas. A continuación, se puede observar una tabla con las películas calificadas.

In [ ]:
RS = RecommenderSystem(DF)

user = "Felipe"
films = ["rear window", "birds", "godfather", "goodfellas", "pulp", "stand by", "cape", "se7en", "boogie", "casino", "pretty", "innocence", "gilbert", "professional"]
scores = [4, 4, 5, 5, 3, 4, 4, 5, 2, 5, 3, 3, 4, 3]
RS.add_user(LM, user, films, scores)


&ensp; Posteriormente, se entrena el filtro colaborativo según lo solicitado en la cátedra y se grafica el costo en función del número de iteraciones.

In [ ]:
iterations = 2000
RS.train_collab_filter(learning_rate=1e-3, lambda_=10, k=10, n_iter=iterations)

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(np.arange(0, iterations), RS.cost)
plt.title("$J(x_i, ~ θ_j)$ vs n iteraciones")
plt.xlabel("Cantidad de Iteraciones")
plt.ylabel("Costo")
plt.grid()
plt.show()

&ensp; En el gráfico se puede observar que, alrededor de la iteración $125$, el costo disminuye considerablemente en relación con su valor inicial. A partir de ese punto, se estabiliza en un valor cercano a los $73000$, manteniéndose prácticamente constante hasta alcanzar las $2000$ iteraciones.

&ensp; Por último, se le recomiendan al usuario $5$ películas, según las calificadas anteriormente. De esta manera, se tiene la siguiente tabla:

In [ ]:
RS.recommend(user, n=5, p=0.5)

---
---

<div style="text-align: center;">
  <h3> Conclusiones
</div>

&ensp; El trabajo permitió implementar un sistema completo de recomendación de películas, combinando distintas técnicas vistas en las clases del curso. La utilización de *embeddings* pre-entrenados mediante $\texttt{GloVe}$ resultó efectiva para capturar similitudes entre títulos de películas, logrando un buscador que soporte títulos incompletos o errores.

&ensp; Por otro lado, el sistema de recomendación basado en un filtro colaborativo permitió identificar preferencias del usuario a partir de un conjunto de calificaciones. El entrenamiento mediante *gradient descent* resultó en parámetros útiles que, combinándolos con las calificaciones promedio, permitieron determinar un *rating* para realizar recomendaciones personalizadas al usuario.

&ensp; Además, el gráfico del costo en función del número de iteraciones, demostró visualmente cómo a mayor cantidad de iteraciones, el método del gradiente descendiente permite minimizar el costo.

&ensp; Como conclusión principal, tanto el modelo de lenguaje como el sistema de recomendación conforman un bloque capaz de interpretar búsquedas y ofrecer sugerencias, destacándose así por la coherencia de las recomendaciones obtenidas y la solidez del desarrollo implementado.